![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/llama.cpp/AutoGGUFVisionModel_text-only_AutoGGUFModel.ipynb)

# Running AutoGGUFVisionModel as text-only AutoGGUFModel

This notebook will show you how you can easily load a vision language model (VLM) for text only inference. This way you can run your favorite VLM for your text tasks!

Let's keep in mind a few things before we start 😊

- Cross-compatibility between `AutoGGUFModel` and `AutoGGUFVisionModel` support was introduced in `Spark NLP 6.1.1`. Please make sure you have upgraded to the latest Spark NLP release.

## Import and Save AutoGGUF models in Spark NLP

- Let's install and setup Spark NLP (if running it Google Colab)
- This part is pretty easy via our simple script

In [ ]:
# Only execute this if you are on Google Colab
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp

# let's start Spark with Spark NLP with GPU enabled. If you don't have GPUs available remove this parameter.
spark = sparknlp.start(gpu=True)
print(sparknlp.version())

6.1.1


25/08/05 10:52:29 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Now let's say we want to use Qwen2.5 VL 3B Q4 from the pretrained `AutoGGUFVisionModel` but only the text model part. 

Doing this is easy: we just have to define the correct pretrained model name and load it into `AutoGGUFModel`. The name of this model is `Qwen2.5_VL_3B_Instruct_Q4_K_M_gguf`. You can find the model in the [Models Hub](https://nlp.johnsnowlabs.com/models).
You will see that the parameters of the model will be set to default values. This is expected, as some vision specific parameters will not be used in this case. 


In [ ]:
from sparknlp.annotator import *

autoGGUFModel = (
    AutoGGUFModel.pretrained("Qwen2.5_VL_3B_Instruct_Q4_K_M_gguf")
    .setInputCols("document")
    .setOutputCol("completions")
    .setBatchSize(4)
    .setNPredict(20)
    .setNGpuLayers(99)
)

Qwen2.5_VL_3B_Instruct_Q4_K_M_gguf download started this may take some time.
Approximate size to download 2.5 GB
[ | ]

Qwen2.5_VL_3B_Instruct_Q4_K_M_gguf download started this may take some time.
Approximate size to download 2.5 GB
[ \ ]Download done! Loading the resource.
[ | ]Failed to load all parameters from file:/home/ducha/cache_pretrained/Qwen2.5_VL_3B_Instruct_Q4_K_M_gguf_en_6.1.1_3.0_1754318104734, attempting fallback loader. Parameters will be set to default values.
Extracted 'libjllama.so' to '/tmp/libjllama.so'
[OK!]


Now let's see how we can use it as usual in a Spark NLP pipeline!

In [ ]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")

pipeline = Pipeline().setStages([document_assembler, autoGGUFModel])

data = spark.createDataFrame([["The moon is "]]).toDF("text")

result = pipeline.fit(data).transform(data)
result.select("completions").show(truncate=False)

+----------------------------------------------------------------------------------------------+
|completions                                                                                   |
+----------------------------------------------------------------------------------------------+
|[{document, 0, 51, The Moon is approximately 384,400 kilometers (238,85, {sentence -> 0}, []}]|
+----------------------------------------------------------------------------------------------+



That's it! You can now go wild and use hundreds of GGUF models from HuggingFace 🤗 in Spark NLP 🚀
